<a href="https://colab.research.google.com/github/wasineer-dev/braid/blob/develop/inputBioPlex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://bioplex.hms.harvard.edu/data/BaitPreyPairs_noFilters_BP2a.tsv

--2023-01-19 01:54:13--  https://bioplex.hms.harvard.edu/data/BaitPreyPairs_noFilters_BP2a.tsv
Resolving bioplex.hms.harvard.edu (bioplex.hms.harvard.edu)... 45.60.80.9
Connecting to bioplex.hms.harvard.edu (bioplex.hms.harvard.edu)|45.60.80.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 621906318 (593M) [text/tab-separated-values]
Saving to: ‘BaitPreyPairs_noFilters_BP2a.tsv’

BaitPreyPairs_noFil 100%[===================>] 593.10M  35.8MB/s    in 17s     

2023-01-19 01:54:31 (34.4 MB/s) - ‘BaitPreyPairs_noFilters_BP2a.tsv’ saved [621906318/621906318]



In [ ]:
!rm -rf braid
!git clone https://github.com/wasineer-dev/braid.git

Cloning into 'braid'...
remote: Enumerating objects: 819, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 819 (delta 102), reused 122 (delta 58), pack-reused 636
Receiving objects: 100% (819/819), 13.17 MiB | 18.38 MiB/s, done.
Resolving deltas: 100% (462/462), done.


In [ ]:
%cd /content/braid
!git switch develop

/content/braid
Branch 'develop' set up to track remote branch 'develop' from 'origin'.
Switched to a new branch 'develop'


In [ ]:
import sys

path = f"/content/braid"
if not path in sys.path:
    sys.path.insert(1, path)

# list all directories in the Python path
print("\n".join(["'" + path + "'" for path in sys.path]))

'/content'
'/content/braid'
'/env/python'
'/usr/lib/python38.zip'
'/usr/lib/python3.8'
'/usr/lib/python3.8/lib-dynload'
''
'/usr/local/lib/python3.8/dist-packages'
'/usr/lib/python3/dist-packages'
'/usr/local/lib/python3.8/dist-packages/IPython/extensions'
'/root/.ipython'


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

from time import time as timer

import importlib
mymodule = importlib.import_module("spoke_model")

import importlib.util
import sys
spec = importlib.util.spec_from_file_location("cpmBioplex", "/content/braid/spoke_model/countBioplexSpokeModel.py")
cbSpokeModel = importlib.util.module_from_spec(spec)
sys.modules["cpmBioplex"] = cbSpokeModel
spec.loader.exec_module(cbSpokeModel)

spec = importlib.util.spec_from_file_location("smlt", "/content/braid/meanfield/simulateLikelihood.py")
smlt = importlib.util.module_from_spec(spec)
sys.modules["smlt"] = smlt
spec.loader.exec_module(smlt)

spec = importlib.util.spec_from_file_location("inputBioplex", "/content/braid/inputFile/inputBioplex.py")
inputBioplex = importlib.util.module_from_spec(spec)
sys.modules["inputBioplex"] = inputBioplex
spec.loader.exec_module(inputBioplex)
            
def clustering(inputSet, Nk, psi, fileName):
    nProteins = inputSet.observationG.nProteins
    cmfa = smlt.CMeanFieldAnnealing(nProteins, Nk) # default

    funcInfer = cmfa

    ts = timer()
    # alpha = 1e-2
    funcInfer.estimate(inputSet.observationG, nProteins, Nk, psi) 
    te = timer()
    print("Time running MFA: ", te-ts)

    funcInfer.find_argmax()
    inputSet.writeCluster2File(fileName, funcInfer.mIndicatorQ, funcInfer.indicatorVec)

bioPlex2 = inputBioplex.CInputBioplex("/content/BaitPreyPairs_noFilters_BP2a.tsv", cbSpokeModel.CountBioplexSpoke)


Number of baits =  5891
Number of preys =  12150
Number of proteins =  12260


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

clustering(bioPlex2, 2000, psi=2.5, fileName="bioplex_2.5_2000.tsv")
clustering(bioPlex2, 3000, psi=2.5, fileName="bioplex_2.5_3000.tsv")
clustering(bioPlex2, 4000, psi=2.5, fileName="bioplex_2.5_4000.tsv")
clustering(bioPlex2, 5000, psi=2.5, fileName="bioplex_2.5_5000.tsv")


psi =  2.5
Initialize with MFA: num. iterations =  10
Time running MFA:  1125.5291023254395
psi =  2.5
Initialize with MFA: num. iterations =  10
Time running MFA:  1118.7022399902344
psi =  2.5
Initialize with MFA: num. iterations =  10
Time running MFA:  1125.3675329685211
psi =  2.5
Initialize with MFA: num. iterations =  10
Time running MFA:  1131.0644264221191


In [ ]:
MAX_ITERATION = 20

class CMeanFieldAnnealing:

    def __init__(self, Nproteins, Nk):
        self.lstExpectedLikelihood = []
        self.mIndicatorQ = np.zeros((Nproteins, Nk), dtype=float)

    def tf_annealing(self, mix_p, mObservationG, Nproteins, Nk, psi):

        matA = tf.convert_to_tensor(mObservationG.mTrials - mObservationG.mObserved, dtype=tf.float32)
        matB = tf.convert_to_tensor(psi*mObservationG.mObserved, dtype=tf.float32)
        tfArray = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
        for i in range(Nproteins):
            tfArray = tfArray.write(i, self.mIndicatorQ[i])
        
        gamma = 1000.0
        nIteration = 0
        while(nIteration < MAX_ITERATION and gamma > 1.0):
            for i in range(Nproteins):        
                tQ = tfArray.stack()
                fn_out = tf.tensordot(matA[i], tQ, axes=1) 
                fp_out = tf.tensordot(matB[i], 1.0 - tQ, axes=1)

                mLogLikelihood = fn_out + fp_out
                tfArray = tfArray.write(i, tf.nn.softmax(-gamma*mLogLikelihood))

            nIteration += 1
            gamma = gamma - 100.0
        print("Initialize with MFA: num. iterations = ", nIteration)
        self.mIndicatorQ = tfArray.stack().numpy()

    def estimate(self, mObservationG, Nproteins, Nk, psi):
        
        print('psi = ', psi)

        mix_p = (1.0/float(Nk))*np.ones(Nk, dtype=float)
        alpha1 = 1e-8
        for i in range(Nproteins):
            self.mIndicatorQ[i] = np.random.uniform(0.0, 1.0, size=Nk)
            self.mIndicatorQ[i] = (self.mIndicatorQ[i] + alpha1)/(np.sum(self.mIndicatorQ[i]) + alpha1*Nproteins)

        with tf.device('/device:GPU:0'):
          self.tf_annealing(mix_p, mObservationG, Nproteins, Nk, psi)

    def find_argmax(self):
        N = np.size(self.mIndicatorQ, axis=0)
        k = np.size(self.mIndicatorQ, axis=1)
        self.indicatorVec = np.argmax(self.mIndicatorQ, axis=1)
        
    def computeErrorRate(self, mObservationG, Nproteins):
        
        self.find_argmax()

        nClusters = len(np.unique(self.indicatorVec))
        print("Number of clusters used: " + str(nClusters))

        countFn = 0
        countFp = 0
        sumSameCluster = 0
        sumDiffCluster = 0
        for i in range(Nproteins):
            for j in mObservationG.lstAdjacency[i]:
                t = mObservationG.mTrials[i][j]
                s = mObservationG.mObserved[i][j]
                assert(s <= t)
                if (self.indicatorVec[i] == self.indicatorVec[j]):
                    countFn += (t - s)
                    sumSameCluster += t
                else:
                    countFp += s
                    sumDiffCluster += t

        counts = countFn + countFp
        fn = 0.0
        fp = 0.0
        if (sumSameCluster > 0):
            fn = float(countFn)/float(sumSameCluster)
        if (sumDiffCluster > 0):
            fp = float(countFp)/float(sumDiffCluster)
        likelihood = countFn*(-np.log(fn) + np.log(1.0 - fp)) + countFp*(-np.log(fp) + np.log(1.0 - fn)) 
        for i in range(Nproteins):
            for j in mObservationG.lstAdjacency[i]:
                t = mObservationG.mTrials[i][j]
                s = mObservationG.mObserved[i][j]
                likelihood += -s*(np.log(1.0-fn)) - (t-s)*(np.log(1.0-fp))
        return (fn, fp, counts, likelihood)